In [167]:
import pyodbc
import pandas as pd

In [168]:
# importar los dos excel para agregar tablas a la BD

rutaarchivo= 'Colocar ruta del acrhivo .xlsx
hoja_estudiantes = 'Poblacion EstudiantilVAL'
hoja_programas = 'programas'

df_estudiantes=pd.read_excel(rutaarchivo, sheet_name=hoja_estudiantes)
df_programas=pd.read_excel(rutaarchivo, sheet_name=hoja_programas)

In [169]:
# crear conexion a BD
server = '127.0.0.1'
database = 'master'
username = 'sa'
password = '12345678'

# se establece la conexion con sqlserver
conn_str = f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}"
conn = pyodbc.connect(conn_str,autocommit=True)

# se crea el cursor controlador
cur = conn.cursor()

In [170]:
# crear la base de datos actividadETL

db_name = "actividadETL"

#funcion para verificar si la base de datos existe


try:
    # Verificar si hay transacciones activas y hacer rollback si es necesario
    cur.execute("IF @@TRANCOUNT > 0 ROLLBACK TRANSACTION")

    # Consultar si la base de datos existe
    cur.execute(f"SELECT name FROM sys.databases WHERE name = '{db_name}'")
    resultado = cur.fetchone()

    if resultado:
        print(f"La base de datos '{db_name}' ya existe.")
    else:
        cur.execute(f"CREATE DATABASE {db_name}")
        print(f"Base de datos '{db_name}' creada.")

    # Confirmar cambios
    conn.commit()

except Exception as e:
    print(f"No se pudo crear la base de datos: {e}")
    conn.rollback()



La base de datos 'actividadETL' ya existe.


In [ ]:
# Cambiar de base de datos master a actividadETL

cur.close()
conn.close() # se cierran la conexion y el cursor

In [ ]:
# Conectarse a actividadETL
conn_str = f"DRIVER={{SQL Server}};SERVER={server};DATABASE={db_name};UID={username};PWD={password}"
conn = pyodbc.connect(conn_str)
cur = conn.cursor()

In [ ]:

# crear tabla estudiantes
tabla = "estudiantes"

try:
    # Verificar si la tabla existe
    cur.execute(f"SELECT COUNT(*) FROM sys.tables WHERE name = '{tabla}'")
    resultado = cur.fetchone()

    if resultado[0] > 0:
        print(f"La tabla '{tabla}' ya existe.")
    else:
        # SQL para crear tabla estudiantes
        
        cur.execute(f"""
            CREATE TABLE {tabla} (
                [Periodo Reg] VARCHAR(100) NOT NULL,
                cohorte VARCHAR(100) NOT NULL,
                Periodo_Academico VARCHAR(100) NOT NULL,
                Tipo_periodo VARCHAR(100) NOT NULL,
                Año INT NOT NULL,
                Numero_Identificacion VARCHAR(100) NOT NULL,
                Codigo_Estudiante INT NOT NULL,
                Cohorte1 VARCHAR(100) NOT NULL,
                Tipo_Acceso VARCHAR(100) NOT NULL,
                Nombre VARCHAR(100) NOT NULL,
                Genero VARCHAR(100) NOT NULL,
                Estado_Civil VARCHAR(100) NOT NULL,
                Estrato_Economico VARCHAR(100) NOT NULL,
                [llave percod] VARCHAR (100) PRIMARY KEY NOT NULL           
            )
        """)
        print(f"Tabla '{tabla}' creada exitosamente.")
    conn.commit() # confirmar los cambios
except Exception as e:
    print(f"Error al verificar o crear la tabla: {e}")
    conn.rollback()    
    


   

Tabla 'estudiantes' creada exitosamente.


In [ ]:


# Insertar los datos del DataFrame estudiantes en la tabla estudiantes
try:
    for index, row in df_estudiantes.iterrows():
        sql=f"""INSERT INTO {tabla} ([Periodo Reg], cohorte, Periodo_Academico, Tipo_periodo, Año, Numero_Identificacion, Codigo_Estudiante, 
            Cohorte1, Tipo_Acceso, Nombre, Genero, Estado_Civil, Estrato_Economico, [llave percod]) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?) """
        cur.execute(sql,
            row['Periodo Reg'], row['cohorte'], row['Periodo_Academico'], row['Tipo_periodo'], row['Año'],row['Numero_Identificacion'],
            row['Codigo_Estudiante'],row['Cohorte'],row['Tipo_Acceso'],row['Nombre'],row['Genero'],row['Estado_Civil'],
            row['Estrato_Economico'], row['llave percod']
        )

    conn.commit()  # Confirmar los cambios
    print(f"{len(df_estudiantes)} registros insertados en '{tabla}'.")

except Exception as e:
    print(f"Error en la fila {index}: {row.to_dict()}")
    print(f"Error al crear la tabla o insertar datos: {e}")
    conn.rollback()

184990 registros insertados en 'estudiantes'.


In [ ]:
# crear tabla programas
tabla = "programas"

try:
    # Verificar si la tabla existe
    cur.execute(f"SELECT COUNT(*) FROM sys.tables WHERE name = '{tabla}'")
    resultado = cur.fetchone()

    if resultado[0] > 0:
        print(f"La tabla '{tabla}' ya existe.")
    else:
        # SQL para Crear la tabla programas
        
        cur.execute(f"""
            CREATE TABLE {tabla} (
                Periodo_Academico VARCHAR(100) NOT NULL,
                Codigo_Estudiante INT NOT NULL,
                Programa VARCHAR(100) NOT NULL,
                Codigo_SNIES INT NOT NULL,
                Ciclo VARCHAR(100) NOT NULL,
                Facultad VARCHAR(100) NOT NULL,
                Nivel_de_Formacion VARCHAR(100) NOT NULL,
                [llave percod] VARCHAR(100) PRIMARY KEY NOT NULL,    
                FOREIGN KEY ([llave percod]) REFERENCES estudiantes([llave percod])    
            )
        """)
        print(f"Tabla '{tabla}' creada exitosamente.")
    conn.commit() # confirmar los cambios
except Exception as e:
    print(f"Error al verificar o crear la tabla: {e}")
    conn.rollback()

Tabla 'programas' creada exitosamente.


In [ ]:
# Insertar los datos del DataFrame programas en la tabla programas
try:
    for index, row in df_programas.iterrows():
        sql=f"""INSERT INTO {tabla} (Periodo_Academico, Codigo_Estudiante, Programa, Codigo_SNIES, Ciclo, Facultad, Nivel_de_Formacion, [llave percod]) 
        VALUES (?, ?, ?, ?, ?, ?, ?, ?) """
        cur.execute(sql,
            row['Periodo_Academico'], row['Codigo_Estudiante'], row['Programa'],row['Codigo_SNIES'], row['Ciclo'],row['Facultad'],row['Nivel_de_Formacion'],
            row['llave percod']
        )

    conn.commit()  # Confirmar los cambios
    print(f"{len(df_programas)} registros insertados en '{tabla}'.")

except Exception as e:
    print(f"Error en la fila {index}: {row.to_dict()}")
    print(f"Error al crear la tabla o insertar datos: {e}")
    conn.rollback()

184990 registros insertados en 'programas'.


In [ ]:
# Cerrar la conexión
cur.close()
conn.close()